###### TODO

1. Ensure Residual block skip connection is working correctly
2. ensure the shapes of inputs and outputs in conv and residual blocks are what is expected

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.activations import relu
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [2]:
# check tf version and if gpu is being used

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

# this is needed for training to occue on rtx 2080 super as of (2021-01-11)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


<b>Load the data and preprocess it, as mnist consists of greyscale images with one channel we will simply convert them to binary so that they are easier to work with

In [3]:
(ds_train, ds_test), ds_info = tfds.load('mnist', split=['test', 'test'], shuffle_files=True,
                                         as_supervised=True, with_info=True)

def to_binary(img, label):
    ''' make images binary '''
    img = tf.cast(img, tf.float32)
    img = tf.math.round(img/255.0)
    return img, tf.cast(img, tf.int32)

ds_train = ds_train.map(to_binary)
ds_train = ds_train.cache()

ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(64)
ds_test = ds_test.map(to_binary).batch(64).cache().prefetch(64)

### Custom Layers

The PixelCNN will require 2 blocks of layers.

1. The Masked 2D Convolution: This is used nsure that the kernel does not have informaion on the value i is predicting. This is done by setting parts of the kernel to zero (masking). There are two types of mask, Type A and B. The type A mask is where the center pixel of the kernelis also masked, in  future layers they are not. Reasoning begind this can be founnd in the paper linked in the readme.


2. The Residual Blocks: These layers will allow the netwprk to optimize the residuals of the input features to make the network more efficient.


The custom layers will be built by subclassing the Layer class in <i>tensorflow.keras.layers.Layer</i>

In [4]:
class MaskedConv2D(layers.Layer):
    def __init__(self, mask_type, kernel=5, filters=1):
        super(MaskedConv2D, self).__init__()
        self.kernel = kernel
        self.filters = filters
        self.mask_type = mask_type
        
        pass
    
    def build(self, input_shape):
        self.w = self.add_weight(shape=[self.kernel, self.kernel, input_shape[-1], self.filters], 
                                 initializer='glorot_normal',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.filters,), initializer='zeros', trainable=True)
        
        mask = np.ones(self.kernel**2, dtype=np.float32)
        center = len(mask) // 2
        if self.mask_type == 'A':
            mask[center] = 0 
            
        mask = mask.reshape((self.kernel, self.kernel, 1, 1))
        
        self.mask = tf.constant(mask, dtype=tf.float32)
        
    def call(self, inputs):
        
        masked_w = tf.math.multiply(self.w, self.mask)
        output = tf.nn.conv2d(inputs, masked_w, 1, "SAME") + self.b
        
        return tf.nn.relu(output)
    
class ResidualBlock(layers.Layer):
    def __init__(self, h=32):
        super(ResidualBlock, self).__init__()
        
        self.forward = Sequential([MaskedConv2D('B', kernel=1, filters=h),
                                   MaskedConv2D('B', kernel=3, filters=h),
                                   MaskedConv2D('B', kernel=1, filters=2*h)])
        
    def call(self, inputs):
        x = self.forward(inputs)
        return x + inputs


def buildPixelCNN(hidden_features=64, output_features=64, num_resblocks=7):
    inputs = layers.Input(shape=[28, 28, 1])
    x = inputs
    x = MaskedConv2D('A', kernel=7, filters=2*hidden_features)(x)
    for _ in range(num_resblocks):
        x = ResidualBlock(hidden_features)(x)
        
    x = layers.Conv2D(output_features, (1,1), padding='same', activation='relu')(x)
    x = layers.Conv2D(1, (1,1), padding='same', activation='sigmoid')(x)
    
    return tf.keras.Model(inputs=inputs, outputs=x, name="pixelcnn")

pixelcnn = buildPixelCNN()
pixelcnn.summary()

Model: "pixelcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
masked_conv2d (MaskedConv2D) (None, 28, 28, 128)       6400      
_________________________________________________________________
residual_block (ResidualBloc (None, 28, 28, 128)       53504     
_________________________________________________________________
residual_block_1 (ResidualBl (None, 28, 28, 128)       53504     
_________________________________________________________________
residual_block_2 (ResidualBl (None, 28, 28, 128)       53504     
_________________________________________________________________
residual_block_3 (ResidualBl (None, 28, 28, 128)       53504     
_________________________________________________________________
residual_block_4 (ResidualBl (None, 28, 28, 128)       535

<b>Training the model</b>

While compiling the model we use Binary Cross Entropy because each call to predict is predicting the value of a pixel, which
can either be 0 or 1. We could use softmax to predict the pixel value of RGB images where N = 3, where N would be the number
of channels.

In [5]:
pixelcnn.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                 optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                 metrics=[tf.keras.losses.BinaryCrossentropy()])

pixelcnn.fit(ds_train, epochs=50, validation_data=ds_test)

Epoch 1/50
157/157 [==============================] - 29s 79ms/step - loss: 0.1551 - binary_crossentropy: 0.1551 - val_loss: 0.0019 - val_binary_crossentropy: 0.0019
Epoch 2/50
157/157 [==============================] - 11s 73ms/step - loss: 0.0228 - binary_crossentropy: 0.0228 - val_loss: 1.1557e-06 - val_binary_crossentropy: 1.1553e-06
Epoch 3/50
157/157 [==============================] - 11s 71ms/step - loss: 0.0295 - binary_crossentropy: 0.0295 - val_loss: 3.4210e-07 - val_binary_crossentropy: 3.4336e-07
Epoch 4/50
157/157 [==============================] - 11s 71ms/step - loss: 5.5180e-04 - binary_crossentropy: 6.2450e-04 - val_loss: 0.1244 - val_binary_crossentropy: 0.1244
Epoch 5/50
157/157 [==============================] - 11s 71ms/step - loss: 0.0050 - binary_crossentropy: 0.0050 - val_loss: 2.1057e-07 - val_binary_crossentropy: 2.1308e-07
Epoch 6/50
157/157 [==============================] - 11s 71ms/step - loss: 2.3883e-08 - binary_crossentropy: 2.3883e-08 - val_loss: 8.753

157/157 [==============================] - 11s 72ms/step - loss: 6.3932e-12 - binary_crossentropy: 6.3930e-12 - val_loss: 6.6791e-12 - val_binary_crossentropy: 6.6671e-12
Epoch 47/50
157/157 [==============================] - 11s 71ms/step - loss: 4.8756e-12 - binary_crossentropy: 4.8755e-12 - val_loss: 6.5078e-12 - val_binary_crossentropy: 6.4960e-12
Epoch 48/50
157/157 [==============================] - 11s 72ms/step - loss: 6.3616e-12 - binary_crossentropy: 6.3612e-12 - val_loss: 6.3446e-12 - val_binary_crossentropy: 6.3331e-12
Epoch 49/50
157/157 [==============================] - 11s 71ms/step - loss: 5.0966e-12 - binary_crossentropy: 5.0967e-12 - val_loss: 6.1889e-12 - val_binary_crossentropy: 6.1776e-12
Epoch 50/50
157/157 [==============================] - 11s 71ms/step - loss: 4.6293e-12 - binary_crossentropy: 4.6292e-12 - val_loss: 6.0424e-12 - val_binary_crossentropy: 6.0314e-12


Use the trained model to generate digits

In [6]:
# grid_row = 5
# grid_col = 5
# batch = grid_row * grid_col
# h = w = 28

# for row in range(h):
#     for col in range(w):
#         prob = pixel_cnn.predict(images)[:, row, col]


In [14]:
# i is the input to the network
img = np.ones((28, 28, 1), dtype=np.float32)

In [ ]:
for row in range(28):
    for col in range(28):
        prob = pixelcnn.predict(img)[row, col, 0]

In [27]:
pixel

(28, 28, 1)

In [ ]:
pixel_cnn.

In [21]:
r = pixelcnn.predict(img)

In [28]:
r.shape

(28, 28, 1, 1)

In [ ]:
# to make generate images with this model we will be "predicting" the value of each pixel one by one.
# 